In [3]:
import numpy as np

In [4]:
import data_loading

In [10]:
percentile_rank_chars, regular_chars, chars, dates,\
    return_panel, permnos, rts, monthly_updates = data_loading.get_data_panel("../Data/raw_chars_returns_df_all_dates_yearly_fb.fthr",
                                                            "../Data/ff_rf.csv",
                                                                              computstat_data_present_filter=True,
                                                                              financial_firm_filter=False)

Index(['A2ME', 'AC', 'AT', 'ATO', 'B2M', 'BETA_d', 'BETA_m', 'C2A', 'CF2B',
       'CF2P', 'CTO', 'D2A', 'D2P', 'DPI2A', 'E2P', 'FC2Y', 'HIGH52', 'INV',
       'IdioVol', 'LEV', 'ME', 'NI', 'NOA', 'OA', 'OL', 'OP', 'PCM', 'PM',
       'PROF', 'Q', 'R12_2', 'R12_7', 'R2_1', 'R36_13', 'R60_13', 'RNA', 'ROA',
       'ROE', 'RVAR', 'S2P', 'SGA2S', 'SPREAD', 'SUV', 'TURN', 'VAR', 'return',
       'date', 'permno', 'monthly_update'],
      dtype='object')
['A2ME' 'AC' 'AT' 'ATO' 'B2M' 'BETA_d' 'BETA_m' 'C2A' 'CF2B' 'CF2P' 'CTO'
 'D2A' 'D2P' 'DPI2A' 'E2P' 'FC2Y' 'HIGH52' 'INV' 'IdioVol' 'LEV' 'ME' 'NI'
 'NOA' 'OA' 'OL' 'OP' 'PCM' 'PM' 'PROF' 'Q' 'R12_2' 'R12_7' 'R2_1'
 'R36_13' 'R60_13' 'RNA' 'ROA' 'ROE' 'RVAR' 'S2P' 'SGA2S' 'SPREAD' 'SUV'
 'TURN' 'VAR']


  0%|          | 0/648 [00:00<?, ?it/s]

In [139]:
flat_rank_chars = percentile_rank_chars.reshape(percentile_rank_chars.shape[0] * percentile_rank_chars.shape[1], 45)
flat_rank_chars = flat_rank_chars[np.any(~np.isnan(flat_rank_chars), axis=1)]

In [140]:
def get_ordered_missing_mat(ct):
    missing_0 = np.isnan(ct)
    missing_counts_by_stock = np.sum(missing_0, axis=1)
    missing_counts_by_char = np.sum(missing_0, axis=0)
    stock_inds_orderd_by_missing = np.array([x[1] for x in sorted([(x, i) for 
                                                                   i, x in enumerate(missing_counts_by_stock)])])

    char_inds_orderd_by_missing = np.array([x[1] for x in sorted([(x, i) for 
                                                                  i, x in enumerate(missing_counts_by_char)])])
    return stock_inds_orderd_by_missing, char_inds_orderd_by_missing

In [141]:
get_ordered_missing_mat(flat_rank_chars)

(array([239013, 239802, 241025, ..., 562257, 566901, 571561]),
 array([12, 20, 32, 41, 38, 44, 18, 43, 42, 31,  9, 14, 30, 39,  4,  8,  0,
         2, 29, 28, 27, 26,  6,  7, 24, 37, 10,  3, 19, 22, 36, 35, 25, 21,
        17,  5, 33, 11, 15, 40, 23,  1, 16, 34, 13]))

In [6]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import copy




In [11]:
fully_present_mask = np.all(~np.isnan(percentile_rank_chars), axis=2)

In [143]:
ordering = [12, 20, 32, 41, 38, 44, 18, 43, 42, 31,  9, 14, 30, 39,  4,  8,  0,
         2, 29, 28, 27, 26,  6,  7, 24, 37, 10,  3, 19, 22, 36, 35, 25, 21,
        17,  5, 33, 11, 15, 40, 23,  1, 16, 34, 13]

In [145]:
train_data = (percentile_rank_chars[:-300,:,ordering], return_panel[:-300], fully_present_mask[:-300])
val_data = (percentile_rank_chars[-300:-100,:,ordering], return_panel[-300:-100], fully_present_mask[-300:-100])
test_data = (percentile_rank_chars[-100:,:,ordering], return_panel[-100:], fully_present_mask[-100:])
train_loader, test_loader, val_loader = get_train_val_test_loaders(train_data, val_data, test_data)

In [146]:
# Neural Autoregressive Density Estimator (NADE)
# Author: Ameya Daigavane

import torch
import torch.nn as nn
import torch.distributions as dist

class Beta_NADE(nn.Module):

    # Initialize.
    def __init__(self, inp_dimensions=45, latent_dimensions=5):
        super(Beta_NADE, self).__init__()
        self.inp_dimensions = inp_dimensions
        self.latent_dimensions = latent_dimensions
        self.hidden_beta = nn.Linear(inp_dimensions, latent_dimensions)
        self.alpha_weights_beta = torch.rand(inp_dimensions, latent_dimensions, requires_grad=True)
        self.alpha_bias_beta = torch.rand(inp_dimensions, requires_grad=True)
        self.beta_scale = 100
        
        self.hidden_alpha = nn.Linear(inp_dimensions, latent_dimensions)
        self.alpha_weights_alpha = torch.rand(inp_dimensions, latent_dimensions, requires_grad=True)
        self.alpha_bias_alpha = torch.rand(inp_dimensions, requires_grad=True)
        self.alpha_scale = 100
        

        # Helper matrix to compute prefix sums of dot-products for the forward pass.
        self.sum_matrix = torch.ones(inp_dimensions, inp_dimensions, requires_grad=False)
        for rownum, row in enumerate(self.sum_matrix):
            row[rownum:] = 0
    
    # For a given input x, obtain the mean vectors describing the Bernoulli distributions for each dimension, and each sample.
    def shape_vectors(self, x):
        # Expand each sample as a diagonal matrix.
        x_diag = torch.stack([torch.diag(x_j) for x_j in x])

        # Compute xi*Wi + bi for each dimension in each sample.
        beta_dot_products = self.hidden_beta(x_diag)

        # Sigmoids of prefix sums of above to get hidden activations.
        hidden_beta_activations = torch.sigmoid(torch.matmul(self.sum_matrix, beta_dot_products))

        # Then multiply element-wise with alpha to get mean vectors.
        beta_out= torch.sigmoid(torch.sum(torch.mul(hidden_beta_activations, self.alpha_weights_beta), dim=2)
                             + self.alpha_bias_beta)
        
        
        # Compute xi*Wi + bi for each dimension in each sample.
        alpha_dot_products = self.hidden_beta(x_diag)

        # Sigmoids of prefix sums of above to get hidden activations.
        hidden_alpha_activations = torch.sigmoid(torch.matmul(self.sum_matrix, alpha_dot_products))

        # Then multiply element-wise with alpha to get mean vectors.
        alpha_out = torch.sigmoid(torch.sum(torch.mul(hidden_alpha_activations, self.alpha_weights_alpha), dim=2)
                             + self.alpha_bias_alpha)
        
        return alpha_out * self.alpha_scale, beta_out * self.beta_scale
    

    # Forward pass to compute log-likelihoods for each input separately.
    def forward(self, x):
        # Obtain mean vectors.
        alphas, betas = self.shape_vectors(x)
        

#         assert np.all(alphas.detach().numpy() > 0)
        
#         assert np.all(x.detach().numpy() > 0)
#         assert np.all(x.detach().numpy() < 1)
        # Compute log-likelihoods using the mean vectors.
        log_likelihoods = (alphas - 1) * torch.log(x) + (betas - 1) * torch.log(1-x)
#         assert ~np.any(np.isnan(log_likelihoods.detach().numpy()))
        log_likelihoods += torch.lgamma(alphas + betas) - torch.lgamma(alphas) - torch.lgamma(betas)
#         assert ~np.any(np.isnan(log_likelihoods.detach().numpy()))
        
        return torch.sum(log_likelihoods, dim=1)
    
    def impute(self, x):
        # TODO add imputation for missing data
        alphas, betas = self.shape_vectors(x)
        return alphas / (alphas + betas)

    # Sample.
    def sample(self, num_samples):
        samples = torch.zeros(num_samples, self.inp_dimensions)
        for sample_num in range(num_samples):
            sample = torch.zeros(self.inp_dimensions)
            for dim in range(self.inp_dimensions):
                h_dim = torch.sigmoid(self.hidden(sample))
                bernoulli_mean_dim = torch.sigmoid(self.alpha_weights[dim].dot(h_dim) + self.alpha_bias[dim])
                distribution = dist.bernoulli.Bernoulli(probs=bernoulli_mean_dim)
                sample[dim] = distribution.sample()
            samples[sample_num] = sample
        return samples

In [147]:
X, Y, ind = train_loader
X = X.float()

X = (X + 0.5) * 0.95 + 0.025

model = Beta_NADE()
# model = model.float()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for i in tqdm(range(50)):
    ll = -1 * model(X).mean()
    if i % 5 == 0:
        print(ll)
        res = torch.sqrt(torch.mean(torch.square(model.impute(X_val) - X_val), axis=0))
        print(res)
        print(torch.mean(res))
        print()
    ll.backward()
    optimizer.step()

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(1424.0934, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2724, 0.2672, 0.2609, 0.2719, 0.2685, 0.2659, 0.2477, 0.2784,
        0.2638, 0.2349, 0.2372, 0.2421, 0.2801, 0.2623, 0.2626, 0.2757, 0.2636,
        0.2630, 0.2623, 0.2795, 0.2592, 0.2682, 0.2742, 0.2511, 0.2579, 0.2289,
        0.2144, 0.2713, 0.2619, 0.2733, 0.2708, 0.2701, 0.2730, 0.2796, 0.2548,
        0.2728, 0.2754, 0.2620, 0.2641, 0.2591, 0.2574, 0.2792, 0.2617, 0.2603],
       grad_fn=<SqrtBackward>)
tensor(0.2624, grad_fn=<MeanBackward0>)



 10%|█         | 5/50 [00:41<05:50,  7.79s/it]

tensor(1297.2656, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2724, 0.2671, 0.2591, 0.2717, 0.2693, 0.2660, 0.2474, 0.2797,
        0.2636, 0.2340, 0.2358, 0.2421, 0.2807, 0.2623, 0.2615, 0.2757, 0.2636,
        0.2641, 0.2638, 0.2790, 0.2590, 0.2733, 0.2787, 0.2525, 0.2550, 0.2307,
        0.2151, 0.2784, 0.2621, 0.2738, 0.2725, 0.2701, 0.2731, 0.2809, 0.2593,
        0.2753, 0.2761, 0.2629, 0.2699, 0.2513, 0.2577, 0.2811, 0.2621, 0.2616],
       grad_fn=<SqrtBackward>)
tensor(0.2630, grad_fn=<MeanBackward0>)



 20%|██        | 10/50 [01:21<05:12,  7.82s/it]

...

 80%|████████  | 40/50 [05:31<01:20,  8.03s/it]

tensor(1027.9718, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2648, 0.2421, 0.2700, 0.2819, 0.2649, 0.2446, 0.2804,
        0.2673, 0.2423, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2636, grad_fn=<MeanBackward0>)



 90%|█████████ | 45/50 [06:15<00:42,  8.42s/it]

tensor(1025.5715, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2646, 0.2404, 0.2702, 0.2828, 0.2648, 0.2446, 0.2804,
        0.2674, 0.2424, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2636, grad_fn=<MeanBackward0>)



100%|██████████| 50/50 [06:56<00:00,  8.33s/it]


In [133]:
torch.mean(torch.sqrt(torch.mean(torch.square(X_val - 0.5), axis=0)))

tensor(0.2617)

In [148]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for i in tqdm(range(50)):
    ll = -1 * model(X).mean()
    if i % 5 == 0:
        print(ll)
        res = torch.sqrt(torch.mean(torch.square(model.impute(X_val) - X_val), axis=0))
        print(res)
        print(torch.mean(res))
        print()
    ll.backward()
    optimizer.step()

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(1023.6982, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2644, 0.2390, 0.2704, 0.2835, 0.2648, 0.2446, 0.2803,
        0.2674, 0.2424, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2635, grad_fn=<MeanBackward0>)



 10%|█         | 5/50 [00:42<06:15,  8.34s/it]

tensor(1022.3522, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2643, 0.2379, 0.2706, 0.2839, 0.2648, 0.2446, 0.2803,
        0.2674, 0.2424, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2635, grad_fn=<MeanBackward0>)



 20%|██        | 10/50 [01:22<05:15,  7.90s/it]

...

 80%|████████  | 40/50 [06:05<01:47, 10.73s/it]

tensor(1016.6048, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2646, 0.2342, 0.2712, 0.2847, 0.2648, 0.2447, 0.2803,
        0.2674, 0.2424, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2635, grad_fn=<MeanBackward0>)



 90%|█████████ | 45/50 [07:10<01:00, 12.10s/it]

tensor(1016.0692, grad_fn=<MulBackward0>)
tensor([0.2455, 0.2725, 0.2647, 0.2340, 0.2712, 0.2847, 0.2648, 0.2447, 0.2803,
        0.2674, 0.2424, 0.2335, 0.2465, 0.2820, 0.2620, 0.2629, 0.2756, 0.2654,
        0.2668, 0.2636, 0.2797, 0.2600, 0.2536, 0.2865, 0.2478, 0.2553, 0.2287,
        0.2197, 0.2934, 0.2614, 0.2717, 0.2767, 0.2704, 0.2729, 0.2903, 0.2513,
        0.2808, 0.2748, 0.2662, 0.2719, 0.2486, 0.2578, 0.2820, 0.2658, 0.2590],
       grad_fn=<SqrtBackward>)
tensor(0.2635, grad_fn=<MeanBackward0>)



100%|██████████| 50/50 [08:22<00:00, 10.06s/it]


In [116]:
loss = nn.MSELoss(reduction='none')


In [102]:
torch.sqrt(mse(model.impute(X), X))

tensor(0.2878, grad_fn=<SqrtBackward>)

In [82]:
X_val, Y_val, ind = val_loader
X_val = X_val.float()

X_val = (X_val + 0.5) * 0.95 + 0.025

In [117]:
mse(model.impute(X_val), X_val).shape

torch.Size([])

In [118]:
torch.sqrt(torch.mean(torch.square(model.impute(X_val) - X_val), axis=0))

tensor([0.3116, 0.2984, 0.2690, 0.2476, 0.2911, 0.2740, 0.2750, 0.2679, 0.3063,
        0.2662, 0.2315, 0.2128, 0.2759, 0.4540, 0.2633, 0.3141, 0.2844, 0.2760,
        0.4000, 0.2619, 0.2804, 0.2878, 0.3503, 0.2814, 0.2381, 0.2558, 0.2488,
        0.2546, 0.3227, 0.2678, 0.2810, 0.2782, 0.2765, 0.3059, 0.2811, 0.2513,
        0.2733, 0.3512, 0.3701, 0.2491, 0.2498, 0.5043, 0.2763, 0.2590, 0.2682],
       grad_fn=<SqrtBackward>)

In [105]:
missing_pattern = np.isnan(percentile_rank_chars).reshape(percentile_rank_chars.shape[0] *
                                                          percentile_rank_chars.shape[1],45)

In [106]:
all_patterns = set()
for i in range(missing_pattern.shape[0]):
    all_patterns.add(tuple(missing_pattern[i]))

In [107]:
len(all_patterns)

14568

In [111]:
2**45

35184372088832